In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import collections

In [2]:
path_book_ratings = "archive/Books_rating_cleaned_new.csv"
br = pd.read_csv(path_book_ratings)

In [3]:
br.shape

(2999990, 3)

In [ ]:
# Drop all rows that contain non-ASCII characters
# Create a boolean mask for rows that contain any non-ASCII characters
mask_non_ascii = br['cleanText'].str.contains(r'[^\x00-\x7F]', regex=True)

# Print how many will be dropped
print(f"Rows containing non-ASCII characters: {mask_non_ascii.sum()}")

# Drop those rows
br_clean = br[~mask_non_ascii].copy()

print(f"✅ Remaining rows after drop: {len(br_clean)}")
br_clean.shape

Rows containing non-ASCII characters: 25548
✅ Remaining rows after drop: 2974442


(2974442, 3)

In [17]:
# replace \' with ' and \" with " "
br_clean['cleanText'] = br_clean['cleanText'].str.replace(r"\\'", "'", regex=True)
br_clean['cleanText'] = br_clean['cleanText'].str.replace(r'\\"', '"', regex=True)

In [18]:
# Identify rows containing escaped apostrophes or double quotes
mask_escaped_apostrophe = br_clean['cleanText'].str.contains(r"\\'", regex=True)
mask_escaped_quote = br_clean['cleanText'].str.contains(r'\\"', regex=True)

# Combine both masks
mask_either = mask_escaped_apostrophe | mask_escaped_quote

# Print summary counts
print(f"Rows with escaped apostrophes (\\'): {mask_escaped_apostrophe.sum()}")
print(f"Rows with escaped double quotes (\\\"): {mask_escaped_quote.sum()}")
print(f"Rows containing either escape sequence: {mask_either.sum()}")

# Optional: Show a few examples
print("\n🔍 Example rows with escaped sequences:")
for i, text in enumerate(br_clean.loc[mask_either, 'cleanText'].head(20)):
    print(f"\n--- Review {i+1} ---")
    print(repr(text))


Rows with escaped apostrophes (\'): 0
Rows with escaped double quotes (\"): 0
Rows containing either escape sequence: 0

🔍 Example rows with escaped sequences:
